<a href="https://colab.research.google.com/github/mohammadhosseinipour/Mistral/blob/main/notebooks_mixtral_8*7B/Mixtral_7*8b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install hqq
!pip install -U deepeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 7.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_c

In [ ]:
# fix numpy in colab
import numpy
from IPython.display import clear_output

# fix triton in colab
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

!git clone https://github.com/dvmazur/mixtral-offloading.git --quiet
!cd mixtral-offloading && pip install -q -r requirements.txt
!huggingface-cli download lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo --quiet --local-dir Mixtral-8x7B-Instruct-v0.1-offloading-demo

clear_output()

In [ ]:
# Step 1: Prepare Mixtral model

# Install and import libraries
import sys
import os

# Assuming 'src' is in the same directory as your current working directory in Colab
# Adjust the path accordingly if 'src' is located elsewhere
sys.path.append(os.path.abspath('./mixtral-offloading'))

import torch
from torch.nn import functional as F
from hqq.core.quantize import BaseQuantizeConfig
from huggingface_hub import snapshot_download
from transformers import AutoConfig, AutoTokenizer
from transformers.utils import logging as hf_logging
from transformers import TextStreamer

# Fix numpy in colab
import numpy
# Add the path where 'src' is located
sys.path.append(os.path.abspath('./mixtral-offloading'))
from src.build_model import OffloadConfig, QuantConfig, build_model

clear_output()


Login to huggingface

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

initialize model

In [ ]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
quantized_model_name = "lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo"
state_path = "Mixtral-8x7B-Instruct-v0.1-offloading-demo"

config = AutoConfig.from_pretrained(quantized_model_name)

device = torch.device("cuda:0")

##### Change this to 5 if you have only 12 GB of GPU VRAM #####
offload_per_layer = 4
# offload_per_layer = 5
###############################################################

num_experts = config.num_local_experts

offload_config = OffloadConfig(
    main_size=config.num_hidden_layers * (num_experts - offload_per_layer),
    offload_size=config.num_hidden_layers * offload_per_layer,
    buffer_size=4,
    offload_per_layer=offload_per_layer,
)


attn_config = BaseQuantizeConfig(
    nbits=4,
    group_size=64,
    quant_zero=True,
    quant_scale=True,
)
attn_config["scale_quant_params"]["group_size"] = 256


ffn_config = BaseQuantizeConfig(
    nbits=2,
    group_size=16,
    quant_zero=True,
    quant_scale=True,
)
quant_config = QuantConfig(ffn_config=ffn_config, attn_config=attn_config)


model = build_model(
    device=device,
    quant_config=quant_config,
    offload_config=offload_config,
    state_path=state_path,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/773 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Loading experts:   0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
past_key_values = None
sequence = None

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_rag_input_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.get_text()

rag_input_data = get_rag_input_data("https://www.unipd.it/en/how-apply")

In [ ]:
# Step 3: Segment the content

def chunk_text(text, chunk_size=512):
    sentences = text.split('. ')
    chunks = []
    current_chunk = []
    current_length = 0
    for sentence in sentences:
        sentence_length = len(sentence.split())
        if current_length + sentence_length > chunk_size:
            chunks.append('. '.join(current_chunk) + '.')
            current_chunk = [sentence]
            current_length = sentence_length
        else:
            current_chunk.append(sentence)
            current_length += sentence_length
    if current_chunk:
        chunks.append('. '.join(current_chunk) + '.')
    return chunks

chunks = chunk_text(rag_input_data)

In [ ]:
# Function to integrate content into the model's context
def integrate_content_into_context(model, tokenizer, content_chunks):
    max_input_length = model.config.max_position_embeddings
    for chunk in content_chunks:
        input_ids = tokenizer(chunk, return_tensors='pt', truncation=True, max_length=max_input_length).input_ids.to(device)
        attention_mask = torch.ones_like(input_ids)
        with torch.no_grad():
            model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                past_key_values=None,
                max_new_tokens=1  # Setting this to a minimal value
            )
# Integrate content into the model
integrate_content_into_context(model, tokenizer, chunks)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
# Step 5: Perform the Evaluation

questions = [
    "Can I apply for the admission of the university of Padua if I am a non-italian student?",
    "Are the courses provided by the university of Padua taught only in Italian?",
    "I missed the deadlines for submitting application for the first semester. Can I enroll and submit the application for the second (winter) semester?",
    "I want to apply for the bachelor's degree in biology, how to know if I am eligible?",
    "The duration of diploma that I obtained is 11 years in my country, does that mean that I can not apply for the admission of the bachelor's degree program in biology of the university of Padua?",
    "How can I make sure if my previous university which is not Italian is qualified in order to apply for the master's degree programs?",
    "Can I apply for just single course units not the whole degree programs of the University of Padua?",
    "My Bachelor's degree was a double degree program obtained from two different universities, one from Indonesia and one from Singapore, how does this affect my application for admission for the master's degree programs?",
    "What are the eligibilities to apply for the admission to a master's degree program?",
    "Can you give the contact info so I can get more detailed academic information?"
]

llm_outputs = []

for question in questions:
    user_entry = dict(role="user", content=question)
    input_ids = tokenizer.apply_chat_template([user_entry], return_tensors="pt").to(device)

    if past_key_values is None:
        attention_mask = torch.ones_like(input_ids)
    else:
        seq_len = input_ids.size(1) + past_key_values[0][0][0].size(1)
        attention_mask = torch.ones([1, seq_len - 1], dtype=torch.int, device=device)

    result = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        past_key_values=None,
        do_sample=True,
        temperature=0.9,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
        return_dict_in_generate=True,
        output_hidden_states=True,
    )

    answer = tokenizer.decode(result["sequences"][0], skip_special_tokens=True)
    llm_outputs.append(answer.strip())

# Ensure we have 10 responses
print(len(llm_outputs))
print(llm_outputs)

#llm_outputs

In [1]:
llm_outputs=["[INST] Can I apply for the admission of the university of Padua if I am a non-italian student? [/INST] Yes, the University of Padua does accept applications from non-Italian students. However, your eligibility to apply will depend on several factors, including your academic background, language proficiency, and visa requirements.\n\nHere are some general steps that you can follow to apply as a non-Italian student:\n\n1. Check the admission requirements: The University of Padua has specific requirements for international students, which you can find on their website. You will need to check if you meet the minimum academic and language proficiency requirements.\n\n2. Choose a program: The University of Padua offers a wide range of programs in various fields, including engineering, humanities, social sciences, and natural sciences. You will need to select the program that you wish to apply for.\n\n3. Prepare your application: You will need to prepare all the necessary documents, such as your academic transcripts, language proficiency scores, and a copy of your passport. Some programs may require additional documents, such as a statement of purpose or letters of recommendation.\n\n4. Submit your application: Once you have prepared all the necessary documents, you can submit your application online through the University of Padua's application portal. There may be an application fee that you need to pay.\n\n5. Wait for the admission decision: After you have submitted your application, you will need to wait for the admission decision. The admission process may take several weeks, so it is important to apply well in advance of the start of the academic year.\n\n6. Apply for a visa: If your application is successful, you will need to apply for a student visa to study in Italy. The visa application process can take several weeks, so it is important to apply as soon as possible.\n\nNote that the specific admission requirements and application procedures may vary depending on your country of origin and the program you are applying for. It is important to carefully review the University of Padua's website and any instructions provided with your application to ensure that you provide all the necessary information and documents.", "[INST] Are the courses provided by the university of Padua taught only in Italian? [/INST] The language of instruction for the majority of courses at the University of Padua is Italian. However, the university does offer some courses in English, particularly at the graduate level and in certain areas such as engineering, economics, and language sciences. It's always a good idea to check the specific language of instruction for each course directly with the university. They also offer language courses for international students who wish to learn or improve their Italian language skills.", '[INST] I missed the deadlines for submitting application for the first semester. Can I enroll and submit the application for the second (winter) semester? [/INST] Whether you can apply for the second (winter) semester depends on the specific policies and procedures of the school or university that you are interested in attending. Some institutions do allow for mid-year admissions and will accept applications for the winter semester, while others have a more rigid academic calendar and do not accept new students during that time.\n\nIf the school or university does allow for winter admissions, you should still expect to go through a similar application process as you would for the fall semester, including meeting the same admissions requirements and deadlines. This may include submitting your transcripts, letters of recommendation, and standardized test scores, as well as completing an application and paying any associated fees.\n\nIt is important to note that if you are applying for the winter semester, you may be competing with a smaller pool of applicants, which could potentially increase your chances of being accepted. However, it is also possible that the school or university may have fewer spots available for winter admissions, so it is important to carefully consider your options and weigh the pros and cons before making a decision.\n\nUltimately, the best way to determine whether you can apply for the winter semester is to contact the school or university directly and inquire about their specific admissions policies and procedures. They should be able to provide you with the information that you need to make an informed decision and move forward with your application.', "[INST] I want to apply for the bachelor's degree in biology, how to know if I am eligible? [/INST] The eligibility criteria for a bachelor's degree program in biology can vary depending on the university or college you are applying to. However, there are some general requirements that are usually common among most institutions. Here are some things to consider:\n\n1. High School Education: You should have completed a high school education from a recognized institution. Some universities may require you to have a minimum GPA or CGPA in your high school education.\n2. Subject Knowledge: It is important that you have a strong background in biology, chemistry, and physics. Some universities may require you to have completed specific courses in these subjects during high school.\n3. Entrance Exams: Some universities may require you to take an entrance exam, such as the SAT or ACT, as part of the application process. You may also be required to take a specific exam for biology or life sciences.\n4. Prerequisites: Some universities may require you to complete certain prerequisites or courses before you can apply for the biology program. This can include completing a specific number of credits in science and mathematics.\n5. Language Proficiency: If you are applying to a university where English is not the primary language, you may be required to provide proof of your English language proficiency.\n\nTo know if you are eligible for a specific bachelor's degree program in biology, you should review the admission requirements of the university or college you are interested in. You can usually find this information on the university's website or by contacting their admissions office. They will be able to provide you with specific information regarding the eligibility criteria and the application process.", "[INST] The duration of diploma that I obtained is 11 years in my country, does that mean that I can not apply for the admission of the bachelor's degree program in biology of the university of Padua? [/INST] The eligibility for admission to a bachelor's degree program in biology at the University of Padua is typically determined by the possession of a valid diploma that gives access to university education in the country where it was obtained. The duration of the diploma is not typically a determining factor for eligibility.\n\nHowever, the University of Padua does have specific requirements for international students who wish to enroll in a degree program, including the submission of all academic transcripts and diplomas, as well as the completion of a successful admission assessment.\n\nIt is recommended that you review the specific admission requirements for the biology degree program at the University of Padua, and that you contact the university's International Students Office or the Biology Department directly for more information. They will be able to provide you with the most up-to-date and accurate information regarding your eligibility for admission.", "[INST] How can I make sure if my previous university which is not Italian is qualified in order to apply for the master's degree programs? [/INST] To determine if your previous university and degree are qualified for a master's degree program in Italy, you should follow these steps:\n\n1. Check the specific requirements of the Italian master's degree program you are interested in. Each university in Italy has the autonomy to set its own admission requirements for international students. Therefore, it is essential to check the admission requirements of the specific master's degree program you want to apply to.\n\n2. Verify the recognition of your degree. The Italian government recognizes foreign degrees that have been awarded by an accredited institution in the country where the degree was granted. However, it is still a good idea to check with the Italian embassy or consulate in your home country to ensure that your degree is recognized.\n\n3. Check the minimum entry requirements for international students in Italy. The minimum entry requirements for international students in Italy include having a valid undergraduate degree from a recognized institution. Therefore, it is essential to ensure that your university and degree are recognized in Italy.\n\n4. Consider seeking advice from an education counselor or advisor. If you are still unsure about the qualifications of your previous university, you can seek advice from an education counselor or advisor. They can help you to understand the Italian education system and the admission requirements for master's degree programs in Italy.\n\n5. Check the equivalent qualifications in the Italian education system. If your degree is not recognized in Italy, you can still apply for a master's degree program by demonstrating equivalent qualifications. For example, if you have a three-year bachelor's degree, you can demonstrate equivalent qualifications by completing a post-graduate diploma or a master's degree that lasts for at least two years.\n\nIn conclusion, to ensure that your previous university is qualified to apply for a master's degree program in Italy, you should check the specific admission requirements of the program, verify the recognition of your degree, check the minimum entry requirements for international students, consider seeking advice from an education counselor or advisor, and check the equivalent qualifications in the Italian education system.", "[INST] Can I apply for just single course units not the whole degree programs of the University of Padua? [/INST] Yes, the University of Padua does offer the option to enroll in individual courses as a Visiting Student, even if you are not pursuing a degree program there. However, the availability of such courses and the application process may vary, so it is best to check the university's website or contact their International Office for the most up-to-date information.\n\nAdditionally, it's important to note that as a Visiting Student, you will not be awarded a degree from the University of Padua, but you will receive a transcript that documents your coursework and grades.\n\nTo apply for Visiting Student status, you will need to submit the necessary documents, including a valid passport, proof of your academic qualifications, and a statement of purpose. You may also be required to provide evidence of your language proficiency in Italian, as many courses are taught in that language.\n\nOnce you are admitted as a Visiting Student, you will be able to register for individual courses, but the availability of courses may be limited and you may need to meet certain prerequisites. Therefore, it is important to plan your course selection in advance and work closely with an academic advisor to ensure that you meet the requirements for enrollment.", "[INST] My Bachelor's degree was a double degree program obtained from two different universities, one from Indonesia and one from Singapore, how does this affect my application for admission for the master's degree programs? [/INST] Having a double degree from two different universities, one from Indonesia and one from Singapore, can be a valuable asset in your application for admission to master's degree programs. This is because a double degree program demonstrates your ability to adapt to new cultural and academic environments, as well as your willingness to take on academic challenges.\n\nHowever, the impact of this on your admission application will depend on several factors, such as the quality of the universities where the degrees were obtained, the fields of study, your academic performance, and your relevant work experience. Admission committees typically consider these factors in order to assess the overall strength of an applicant's profile.\n\nIt's important to highlight the skills and knowledge you acquired during your double degree program, such as your ability to work in diverse teams, your cultural competence, and your in-depth knowledge of two different academic fields. This can be done by emphasizing these skills in your statement of purpose, letters of recommendation, and resume.\n\nUltimately, having a double degree from two different universities can strengthen your application for admission to master's degree programs, but it is only one of many factors that admission committees consider when making their decisions. Therefore, it's essential to present a well-rounded and compelling application that showcases your strengths and potential as a graduate student.", "[INST] What are the eligibilities to apply for the admission to a master's degree program? [/INST] The eligibility to apply for a master's degree program generally depends on the specific requirements of the university and the country where it is located. However, there are some common requirements that most universities look for:\n\n1. Bachelor's degree: A bachelor's degree from a recognized university is the most important requirement. Some universities require the bachelor's degree to be in a related field, while others may accept degrees from any field.\n2. GPA: A minimum undergraduate GPA is required by many universities. The required GPA varies, but it is usually around 3.0 on a 4.0 scale.\n3. Standardized tests: Some universities require the submission of standardized test scores, such as the Graduate Record Examination (GRE) or the Graduate Management Admission Test (GMAT). Some universities may waive this requirement for applicants who meet certain criteria.\n4. Letters of recommendation: Most universities require at least two letters of recommendation from professors or professionals who are familiar with the applicant's work.\n5. Statement of purpose: Applicants are often required to submit a statement of purpose or letter of intent, which outlines their research interests, career goals, and reasons for pursuing a master's degree.\n6. Language proficiency: International students may be required to demonstrate their proficiency in the English language by submitting scores from the Test of English as a Foreign Language (TOEFL) or the International English Language Testing System (IELTS).\n\nIt is important to note that these requirements can vary significantly depending on the university, the country, and the specific master's degree program. Therefore, it is always best to check the specific requirements of the university and program to which you are applying.", "[INST] Can you give the contact info so I can get more detailed academic information? [/INST] I'm happy to help you learn more about various academic topics, but I'm afraid I can't provide you with specific contact information for any institution or organization. However, I can help you find the information you need by suggesting some general steps you can take.\n\n1. Identify the subject or field you're interested in.\n2. Look for universities or colleges with strong programs in that area.\n3. Visit their official websites and explore their academic offerings, research centers, and faculty profiles.\n4. Reach out to the relevant departments or faculty members via their official email addresses for more specific information.\n\nIf you'd like, I can provide some examples of useful resources for general academic inquiries:\n\n- Worldwide University Rankings:\n  - Times Higher Education (THE): <https://www.timeshighereducation.com/world-university-rankings>\n  - QS World University Rankings: <https://www.topuniversities.com/qs-world-university-rankings>\n  - Academic Ranking of World Universities (ARWU): <http://www.shanghaibranking.com/>\n- Subject-specific rankings and information:\n  - US News & World Report: <https://www.usnews.com/education/worlds-universities>\n  - QS Subject Rankings: <https://www.topuniversities.com/student-guide/qs-world-university-rankings/subject/rankings>\n- General academic research:\n  - Google Scholar: <https://scholar.google.com/>\n  - Microsoft Academic: <https://academic.microsoft.com/>\n  - Semantic Scholar: <https://www.semanticscholar.org/>\n\nThese resources should help you find the academic information you're looking for. If you have any questions or need further guidance, please don't hesitate to ask."]

In [25]:
answers=[]
# Extract the text after the [/INST] tag and before the next tag or end of string
for output in llm_outputs:
    answers.append(output.split('[/INST]')[-1].strip())

print(answers)

["Yes, the University of Padua does accept applications from non-Italian students. However, your eligibility to apply will depend on several factors, including your academic background, language proficiency, and visa requirements.\n\nHere are some general steps that you can follow to apply as a non-Italian student:\n\n1. Check the admission requirements: The University of Padua has specific requirements for international students, which you can find on their website. You will need to check if you meet the minimum academic and language proficiency requirements.\n\n2. Choose a program: The University of Padua offers a wide range of programs in various fields, including engineering, humanities, social sciences, and natural sciences. You will need to select the program that you wish to apply for.\n\n3. Prepare your application: You will need to prepare all the necessary documents, such as your academic transcripts, language proficiency scores, and a copy of your passport. Some programs may

In [26]:
print(answers[0])

Yes, the University of Padua does accept applications from non-Italian students. However, your eligibility to apply will depend on several factors, including your academic background, language proficiency, and visa requirements.

Here are some general steps that you can follow to apply as a non-Italian student:

1. Check the admission requirements: The University of Padua has specific requirements for international students, which you can find on their website. You will need to check if you meet the minimum academic and language proficiency requirements.

2. Choose a program: The University of Padua offers a wide range of programs in various fields, including engineering, humanities, social sciences, and natural sciences. You will need to select the program that you wish to apply for.

3. Prepare your application: You will need to prepare all the necessary documents, such as your academic transcripts, language proficiency scores, and a copy of your passport. Some programs may require a

In [27]:
answers[1]

"The language of instruction for the majority of courses at the University of Padua is Italian. However, the university does offer some courses in English, particularly at the graduate level and in certain areas such as engineering, economics, and language sciences. It's always a good idea to check the specific language of instruction for each course directly with the university. They also offer language courses for international students who wish to learn or improve their Italian language skills."

In [28]:
answers[2]

'Whether you can apply for the second (winter) semester depends on the specific policies and procedures of the school or university that you are interested in attending. Some institutions do allow for mid-year admissions and will accept applications for the winter semester, while others have a more rigid academic calendar and do not accept new students during that time.\n\nIf the school or university does allow for winter admissions, you should still expect to go through a similar application process as you would for the fall semester, including meeting the same admissions requirements and deadlines. This may include submitting your transcripts, letters of recommendation, and standardized test scores, as well as completing an application and paying any associated fees.\n\nIt is important to note that if you are applying for the winter semester, you may be competing with a smaller pool of applicants, which could potentially increase your chances of being accepted. However, it is also po

In [29]:
answers[3]

"The eligibility criteria for a bachelor's degree program in biology can vary depending on the university or college you are applying to. However, there are some general requirements that are usually common among most institutions. Here are some things to consider:\n\n1. High School Education: You should have completed a high school education from a recognized institution. Some universities may require you to have a minimum GPA or CGPA in your high school education.\n2. Subject Knowledge: It is important that you have a strong background in biology, chemistry, and physics. Some universities may require you to have completed specific courses in these subjects during high school.\n3. Entrance Exams: Some universities may require you to take an entrance exam, such as the SAT or ACT, as part of the application process. You may also be required to take a specific exam for biology or life sciences.\n4. Prerequisites: Some universities may require you to complete certain prerequisites or cour

In [30]:
answers[4]

"The eligibility for admission to a bachelor's degree program in biology at the University of Padua is typically determined by the possession of a valid diploma that gives access to university education in the country where it was obtained. The duration of the diploma is not typically a determining factor for eligibility.\n\nHowever, the University of Padua does have specific requirements for international students who wish to enroll in a degree program, including the submission of all academic transcripts and diplomas, as well as the completion of a successful admission assessment.\n\nIt is recommended that you review the specific admission requirements for the biology degree program at the University of Padua, and that you contact the university's International Students Office or the Biology Department directly for more information. They will be able to provide you with the most up-to-date and accurate information regarding your eligibility for admission."

In [31]:
answers[5]

"To determine if your previous university and degree are qualified for a master's degree program in Italy, you should follow these steps:\n\n1. Check the specific requirements of the Italian master's degree program you are interested in. Each university in Italy has the autonomy to set its own admission requirements for international students. Therefore, it is essential to check the admission requirements of the specific master's degree program you want to apply to.\n\n2. Verify the recognition of your degree. The Italian government recognizes foreign degrees that have been awarded by an accredited institution in the country where the degree was granted. However, it is still a good idea to check with the Italian embassy or consulate in your home country to ensure that your degree is recognized.\n\n3. Check the minimum entry requirements for international students in Italy. The minimum entry requirements for international students in Italy include having a valid undergraduate degree fro

In [32]:
answers[6]

"Yes, the University of Padua does offer the option to enroll in individual courses as a Visiting Student, even if you are not pursuing a degree program there. However, the availability of such courses and the application process may vary, so it is best to check the university's website or contact their International Office for the most up-to-date information.\n\nAdditionally, it's important to note that as a Visiting Student, you will not be awarded a degree from the University of Padua, but you will receive a transcript that documents your coursework and grades.\n\nTo apply for Visiting Student status, you will need to submit the necessary documents, including a valid passport, proof of your academic qualifications, and a statement of purpose. You may also be required to provide evidence of your language proficiency in Italian, as many courses are taught in that language.\n\nOnce you are admitted as a Visiting Student, you will be able to register for individual courses, but the avai

In [33]:
answers[7]

"Having a double degree from two different universities, one from Indonesia and one from Singapore, can be a valuable asset in your application for admission to master's degree programs. This is because a double degree program demonstrates your ability to adapt to new cultural and academic environments, as well as your willingness to take on academic challenges.\n\nHowever, the impact of this on your admission application will depend on several factors, such as the quality of the universities where the degrees were obtained, the fields of study, your academic performance, and your relevant work experience. Admission committees typically consider these factors in order to assess the overall strength of an applicant's profile.\n\nIt's important to highlight the skills and knowledge you acquired during your double degree program, such as your ability to work in diverse teams, your cultural competence, and your in-depth knowledge of two different academic fields. This can be done by emphas

In [34]:
answers[8]

"The eligibility to apply for a master's degree program generally depends on the specific requirements of the university and the country where it is located. However, there are some common requirements that most universities look for:\n\n1. Bachelor's degree: A bachelor's degree from a recognized university is the most important requirement. Some universities require the bachelor's degree to be in a related field, while others may accept degrees from any field.\n2. GPA: A minimum undergraduate GPA is required by many universities. The required GPA varies, but it is usually around 3.0 on a 4.0 scale.\n3. Standardized tests: Some universities require the submission of standardized test scores, such as the Graduate Record Examination (GRE) or the Graduate Management Admission Test (GMAT). Some universities may waive this requirement for applicants who meet certain criteria.\n4. Letters of recommendation: Most universities require at least two letters of recommendation from professors or p

In [35]:
answers[9]

"I'm happy to help you learn more about various academic topics, but I'm afraid I can't provide you with specific contact information for any institution or organization. However, I can help you find the information you need by suggesting some general steps you can take.\n\n1. Identify the subject or field you're interested in.\n2. Look for universities or colleges with strong programs in that area.\n3. Visit their official websites and explore their academic offerings, research centers, and faculty profiles.\n4. Reach out to the relevant departments or faculty members via their official email addresses for more specific information.\n\nIf you'd like, I can provide some examples of useful resources for general academic inquiries:\n\n- Worldwide University Rankings:\n  - Times Higher Education (THE): <https://www.timeshighereducation.com/world-university-rankings>\n  - QS World University Rankings: <https://www.topuniversities.com/qs-world-university-rankings>\n  - Academic Ranking of W

In [36]:
import os
from getpass import getpass

# Get the API token from the user
OPENAI_API_KEY = getpass("Enter your openai API token: ")

Enter your openai API token: ··········


In [37]:
# Set the environment variable
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [38]:
expected_answers = ["yes, you can. The University of Padua welcomes applications from international students. You are considered an “international student” if you hold a non-Italian academic diploma, degree or equivalent.",
                    "No. There are two type of courses; ENGLISH-taught degree courses and ITALIAN-taught degree courses ",
                    "No, unfortunately you can not. courses at the University of Padua start at the end of September/beginning of October. Enrolment for the 2nd semester is not available.",
                    "To be eligible for admission to a Bachelor’s degree or a single-cycle degree, you need a minimum 12-year school record. If you have been awarded a final diploma after a shorter programme of study (i.e. less than 12 years), in order to become eligible you must: have attended at least 1 (in case of 11 years of schooling) or 2 (in case of 10 years of schooling) years of academic studies and passed all the scheduled exams. OR have obtained a post-secondary qualification  in a non-University higher education institution, certifying the additional one or two years of education. OR have successfully completed one or two year preparatory courses (Foundation courses) to compensate for the missing years of schooling.",
                    "you must take additional courses to become eligible for admission to a Bachelor's degree program. If you have been awarded a final diploma after a shorter programme of study (i.e. less than 12 years), in order to become eligible you must: have attended at least 1 (in case of 11 years of schooling) or 2 (in case of 10 years of schooling) years of academic studies and passed all the scheduled exams. OR have obtained a post-secondary qualification  in a non-University higher education institution, certifying the additional one or two years of education. OR have successfully completed one or two year preparatory courses (Foundation courses) to compensate for the missing years of schooling. Or as an alternative, you must: have attended an additional one (in case of 12 years of schooling) or two (in case of 11 years of schooling) years of academic studies in another country and/or different foreign system and passed all the scheduled exams. OR have obtained an official Italian or foreign post-secondary qualification in a non-University higher education institution with a minimum duration of one or two years. OR have successfully completed Foundation courses held from an Italian University. These additional certificates only allow enrolment to the first year and cannot be further evaluated for shortening of your degree programme",
                    "You can check a country-based list of qualifications in order to verify whether their entry qualification is suitable to access your degree programme of interest.",
                    "yes, you can. If you are you interested in attending single course units and take the corresponding exams without having to be enrolled in any degree course, visit the dedicated page.",
                    "If your bachelor's degree was issued by two or more universities (e.g. the awarding and the teaching institutions are not the same and pertain to different national education systems), each institution must be accredited according to the national educational accreditation system of the country where it is located..",
                    "To be eligible for admission to a Master’s degree, you need a Bachelor’s degree or equivalent foreign qualification with specific curricular requirements obtained from a higher education institution accredited according to the national educational accreditation system of the country where it is located.",
                    "sure,For academic related information you can contact the programme coordinator/secretariat tel. +39 0498273131, or send an email to urp@unipd.it "]

In [39]:
retrieval_contexts = [
    "The University of Padua welcomes applications from international students. You are considered an “international student” if you hold a non-Italian academic diploma, degree or equivalent.",
    "There are two types of degree courses; ENGLISH-taught degree courses: check application procedures and the course factsheets for detailed information. In order to submit your application (except for the Single-cycle degree programme in Medicine and Surgery), please go to: apply.unipd.it ITALIAN-taught degree courses: applications dates vary depending on the course - check application procedures",
    "Courses at the University of Padua start at the end of September/beginning of October. Enrolment for the 2nd semester is not available.",
    "To be eligible for admission to a Bachelor’s degree or a single-cycle degree, you need a minimum 12-year school record. If you have been awarded a final diploma after a shorter programme of study (i.e. less than 12 years), in order to become eligible you must: have attended at least 1 (in case of 11 years of schooling) or 2 (in case of 10 years of schooling) years of academic studies and passed all the scheduled exams. OR have obtained a post-secondary qualification  in a non-University higher education institution, certifying the additional one or two years of education. OR have successfully completed one or two year preparatory courses (Foundation courses) to compensate for the missing years of schooling.",
    "To be eligible for admission to a Bachelor’s degree or a single-cycle degree, you need a minimum 12-year school record. If you have been awarded a final diploma after a shorter programme of study (i.e. less than 12 years), in order to become eligible you must: have attended at least 1 (in case of 11 years of schooling) or 2 (in case of 10 years of schooling) years of academic studies and passed all the scheduled exams. OR have obtained a post-secondary qualification  in a non-University higher education institution, certifying the additional one or two years of education. OR have successfully completed one or two year preparatory courses (Foundation courses) to compensate for the missing years of schooling. As an alternative to this certification, you must: have attended an additional one (in case of 12 years of schooling) or two (in case of 11 years of schooling) years of academic studies in another country and/or different foreign system and passed all the scheduled exams. OR have obtained an official Italian or foreign post-secondary qualification in a non-University higher education institution with a minimum duration of one or two years. OR have successfully completed Foundation courses held from an Italian University. These additional certificates only allow enrolment to the first year and cannot be further evaluated for shortening of your degree programme",
    "Foreign entry qualifications suitable for admission; Applicants can consult a country-based list of qualifications in order to verify whether their entry qualification is suitable to access their degree programme of interest..",
    "Apply for a single course unit ; If you are you interested in attending single course units and take the corresponding exams without having to be enrolled in any degree course, visit the dedicated page.",
    "Should your study title be issued by two or more universities (e.g. the awarding and the teaching institutions are not the same and pertain to different national education systems), each institution must be accredited according to the national educational accreditation system of the country where it is located.",
    "To be eligible for admission to a Master’s degree, you need a Bachelor’s degree or equivalent foreign qualification with specific curricular requirements obtained from a higher education institution accredited according to the national educational accreditation system of the country where it is located.",
    "Global Engagement Office - Recruitment Unit Palazzo Anselmi Lungargine del Piovego 1 - 35131 Padova Get in touch with the Global Engagement Office For academic related information, please contact the programme coordinator/secretariat tel. +39 0498273131 or send an email to  urp@unipd.it "
]


In [40]:
from deepeval import evaluate
from deepeval.metrics.ragas import RagasMetric
from deepeval.test_case import LLMTestCase

In [41]:
actual_outputs = llm_outputs

expected_outputs = expected_answers

test_cases = []
# Do not change the threshold or model for the sake of having similar measurement criteria
metric = RagasMetric(threshold=0.6, model="gpt-4o")

# Loop through each question and create a test case
for idx, (actual_output, expected_output, retrieval_context) in enumerate(zip(actual_outputs, expected_outputs, retrieval_contexts)):
    test_case = LLMTestCase(
        input=f"Question {idx + 1}",
        actual_output=actual_output,
        expected_output=expected_output,
        retrieval_context=[retrieval_context]  # Wrap the string in a list
    )
    test_cases.append(test_case)

# Evaluate test cases in batch
metrics = evaluate(test_cases, [metric])


Output()

Evaluating test cases...
Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]



Metrics Summary

  - ✅ RAGAS (score: 0.6121297503337471, threshold: 0.6, strict: False, evaluation model: gpt-4o, reason: None, error: None)
      - Contextual Precision (ragas) (score: 0.9999999999)
      - Contextual Recall (ragas) (score: 0.6666666666666666)
      - Faithfulness (ragas) (score: 0.03571428571428571)
      - Answer Relevancy (ragas) (score: 0.7461380490540358)

For test case:

  - input: Question 1
  - actual output: [INST] Can I apply for the admission of the university of Padua if I am a non-italian student? [/INST] Yes, the University of Padua does accept applications from non-Italian students. However, your eligibility to apply will depend on several factors, including your academic background, language proficiency, and visa requirements.

Here are some general steps that you can follow to apply as a non-Italian student:

1. Check the admission requirements: The University of Padua has specific requirements for international students, which you can find on their

/usr/local/lib/python3.10/dist-packages/portalocker/utils.py:218: UserWarning: timeout has no effect in blocking mode
  warnings.warn(


✅ Tests finished! Run "deepeval login" to view evaluation results on the web.

In [42]:
from statistics import mean

# puting the results of metrics in test_results to avoid confusion
test_results = metrics
# Initialize lists to collect scores for each metric and overall scores
contextual_precision_scores = []
contextual_recall_scores = []
faithfulness_scores = []
answer_relevancy_scores = []
overall_scores = []

# Iterate through each test result to extract the scores from score_breakdown and overall score
for test_result in test_results:
    # Access the RagasMetric object
    ragas_metric = test_result.metrics[0]

    # Access the score_breakdown dictionary
    score_breakdown = ragas_metric.score_breakdown

    # Append the scores to the respective lists
    contextual_precision_scores.append(score_breakdown['Contextual Precision (ragas)'])
    contextual_recall_scores.append(score_breakdown['Contextual Recall (ragas)'])
    faithfulness_scores.append(score_breakdown['Faithfulness (ragas)'])
    answer_relevancy_scores.append(score_breakdown['Answer Relevancy (ragas)'])

    # Append the overall score to the overall_scores list
    overall_scores.append(ragas_metric.score)

# Calculate the average for each metric
avg_contextual_precision = mean(contextual_precision_scores)
avg_contextual_recall = mean(contextual_recall_scores)
avg_faithfulness = mean(faithfulness_scores)
avg_answer_relevancy = mean(answer_relevancy_scores)
avg_overall_score = mean(overall_scores)

# Print the average scores

print("Mixtral-8x7B-Instruct-v0.1(with offload_per_layer = 4 to fit into memory!) results:")
print(f"Contextual Precision (average score: {avg_contextual_precision})")
print(f"Contextual Recall (average score: {avg_contextual_recall})")
print(f"Faithfulness (average score: {avg_faithfulness})")
print(f"Answer Relevancy (average score: {avg_answer_relevancy})")
print(f"RAGAS (average overall score: {avg_overall_score})")

Mixtral-8x7B-Instruct-v0.1(with offload_per_layer = 4 to fit into memory!) results:
Contextual Precision (average score: 0.9999999999)
Contextual Recall (average score: 0.8333333333333334)
Faithfulness (average score: 0.07240601503759399)
Answer Relevancy (average score: 0.5217583407495905)
RAGAS (average overall score: 0.6068744222551294)
